# 0. input data

In [1]:
day = int(input('day?: '))
time = input('time?: ')
code_int=input('code?: ')

day?:  20231007
time?:  18
code?:  1156054000


In [2]:
alpha=float(input('truncation coeff?(0 ~ 1): '))

truncation coeff?(0 ~ 1):  0.3


# 1.0 stay data

In [3]:
import pandas as pd
import numpy as np

In [4]:
import os
os.chdir('E:\juypter notebook\빅콘 2024')

In [5]:
import data_rev
from preprocessing import stay

In [6]:
df=data_rev.df_revised
df.head()

,name,code,coordinates
0,서울특별시 종로구 청운효자동,1111051500,"[[126.97358726355343, 37.593289023280704], [12..."
1,서울특별시 종로구 사직동,1111053000,"[[126.97688884274817, 37.575650779448786], [12..."
2,서울특별시 종로구 삼청동,1111054000,"[[126.98268938649305, 37.5950655194224], [126...."
3,서울특별시 종로구 부암동,1111055000,"[[126.97585113775686, 37.59656422224409], [126..."
4,서울특별시 종로구 평창동,1111056000,"[[126.97507466788086, 37.63138628651299], [126..."


In [7]:
df2=stay(day, time)

In [8]:
df2['hdong_cd'] = df2['hdong_cd'].astype(str)
df = pd.merge(df, df2, left_on='code', right_on='hdong_cd', how='inner')
df = df.drop(columns=['hdong_cd'])
df = df.rename(columns={'stay_cnts': 'pop'})
df.head()

,name,code,coordinates,pop
0,서울특별시 종로구 청운효자동,1111051500,"[[126.97358726355343, 37.593289023280704], [12...",26334
1,서울특별시 종로구 사직동,1111053000,"[[126.97688884274817, 37.575650779448786], [12...",20805
2,서울특별시 종로구 삼청동,1111054000,"[[126.98268938649305, 37.5950655194224], [126....",10389
3,서울특별시 종로구 부암동,1111055000,"[[126.97585113775686, 37.59656422224409], [126...",12189
4,서울특별시 종로구 평창동,1111056000,"[[126.97507466788086, 37.63138628651299], [126...",16055


In [9]:
seoularea=pd.read_csv('data/seoularea.csv')
seoularea=seoularea[['행정동코드','면적']]
seoularea.columns=['code','area']
seoularea.head()

,code,area
0,1111051500,2.57
1,1111053000,1.23
2,1111054000,1.49
3,1111055000,2.27
4,1111056000,8.87


In [10]:
sc=150000

In [11]:
df['pop'] = df["pop"].fillna(0)
df['scaled_pop']=df['pop'] / sc  ##scale parameter
df['area']=seoularea['area']
df['pop_density']=df['pop']/df['area']
df['scaled_pop_density']=df['pop'] / sc
df.head()

,name,code,coordinates,pop,scaled_pop,area,pop_density,scaled_pop_density
0,서울특별시 종로구 청운효자동,1111051500,"[[126.97358726355343, 37.593289023280704], [12...",26334,0.175560,2.57,10246.692607,0.175560
1,서울특별시 종로구 사직동,1111053000,"[[126.97688884274817, 37.575650779448786], [12...",20805,0.138700,1.23,16914.634146,0.138700
2,서울특별시 종로구 삼청동,1111054000,"[[126.98268938649305, 37.5950655194224], [126....",10389,0.069260,1.49,6972.483221,0.069260
3,서울특별시 종로구 부암동,1111055000,"[[126.97585113775686, 37.59656422224409], [126...",12189,0.081260,2.27,5369.603524,0.081260
4,서울특별시 종로구 평창동,1111056000,"[[126.97507466788086, 37.63138628651299], [126...",16055,0.107033,8.87,1810.033822,0.107033


# 2.0 od data

In [12]:
#central lon, lat 계산
coordinates=df['coordinates'].to_numpy()
lon = np.zeros(426);lat = np.zeros(426)
for i in range(426):
    lonlat=np.mean(coordinates[i],axis=0)
    lon[i],lat[i]=lonlat[0],lonlat[1]

In [13]:
df['lat']=lat
df['lon']=lon
df1=df[['code','lat','lon']]
df1.head()

,code,lat,lon
0,1111051500,37.585281,126.968757
1,1111053000,37.575267,126.969738
2,1111054000,37.586728,126.982915
3,1111055000,37.597352,126.960548
4,1111056000,37.611640,126.970384


In [14]:
arr=df1.to_numpy()
arr_ft=np.zeros((426*426,6))
for i,f in enumerate(arr):
    for j,t in enumerate(arr):
        arr_ft[i*426+j]=np.hstack((f,t))

In [15]:
#관심 행정동 코드
other_coord=list()
for i in arr:
    if i[0]!=code_int: other_coord.append(i)
    else: coordlst=i

In [16]:
coord=coordlst
for i in range(425-1):
    coord=np.vstack((coord,coordlst))

In [17]:
arr_in=np.hstack((other_coord,coord))
arr_out=np.hstack((coord,other_coord))

In [18]:
df_in=pd.DataFrame(arr_in)
df_in.columns = ['from','from_lat','from_lon','to','to_lat','to_lon']
df_in.head()

,from,from_lat,from_lon,to,to_lat,to_lon
0,1111051500,37.585281,126.968757,1156054000,37.526698,126.92512
1,1111053000,37.575267,126.969738,1156054000,37.526698,126.92512
2,1111054000,37.586728,126.982915,1156054000,37.526698,126.92512
3,1111055000,37.597352,126.960548,1156054000,37.526698,126.92512
4,1111056000,37.61164,126.970384,1156054000,37.526698,126.92512


In [19]:
df_out=pd.DataFrame(arr_out)
df_out.columns = ['from','from_lat','from_lon','to','to_lat','to_lon']
df_out.head()

,from,from_lat,from_lon,to,to_lat,to_lon
0,1156054000,37.526698,126.92512,1111051500,37.585281,126.968757
1,1156054000,37.526698,126.92512,1111053000,37.575267,126.969738
2,1156054000,37.526698,126.92512,1111054000,37.586728,126.982915
3,1156054000,37.526698,126.92512,1111055000,37.597352,126.960548
4,1156054000,37.526698,126.92512,1111056000,37.61164,126.970384


In [20]:
def trunc(data,alpha): #alpha is truncation constant: data < alpha => data = 0
    data=data.to_numpy()
    data=data.copy()
    for i in range(len(data)):
        if data[i]-alpha<0: data[i]=0
    return data

In [22]:
from preprocessing import od_in, od_out

In [23]:
in_pop = od_in(date=day, end_time=time, dest=code_int, from_seoul=True)

df_in = pd.merge(df_in, in_pop, how="left", left_on="from", right_on="origin_hdong_cd")
del df_in["origin_hdong_cd"]
df_in.rename(columns={"od_cnts": "pop"}, inplace=True)

In [24]:
sc=150

In [25]:
df_in['pop'] = df_in['pop'].fillna(0)
df_in['scaled_pop']=df_in['pop'] / sc
df_in['trunc_scaled_pop']=trunc(df_in['scaled_pop'],alpha)
df_in.head()

,from,from_lat,from_lon,to,to_lat,to_lon,pop,scaled_pop,trunc_scaled_pop
0,1111051500,37.585281,126.968757,1156054000,37.526698,126.92512,0.0,0.000000,0.000000
1,1111053000,37.575267,126.969738,1156054000,37.526698,126.92512,77.0,0.513333,0.513333
2,1111054000,37.586728,126.982915,1156054000,37.526698,126.92512,5.0,0.033333,0.000000
3,1111055000,37.597352,126.960548,1156054000,37.526698,126.92512,0.0,0.000000,0.000000
4,1111056000,37.61164,126.970384,1156054000,37.526698,126.92512,19.0,0.126667,0.000000


In [26]:
out_pop = od_out(date=day, start_time=time, origin=code_int, to_seoul=True)
df_out = pd.merge(df_out, out_pop, how="left", left_on="to", right_on="dest_hdong_cd")
del df_out["dest_hdong_cd"]
df_out.rename(columns={"od_cnts": "pop"}, inplace=True)

In [27]:
df_out['pop'] = df_out["pop"].fillna(0)
df_out['scaled_pop']=df_out['pop'] / sc
df_out['trunc_scaled_pop']=trunc(df_out['scaled_pop'],alpha)
df_out.head()

,from,from_lat,from_lon,to,to_lat,to_lon,pop,scaled_pop,trunc_scaled_pop
0,1156054000,37.526698,126.92512,1111051500,37.585281,126.968757,0.0,0.000000,0.0
1,1156054000,37.526698,126.92512,1111053000,37.575267,126.969738,5.0,0.033333,0.0
2,1156054000,37.526698,126.92512,1111054000,37.586728,126.982915,0.0,0.000000,0.0
3,1156054000,37.526698,126.92512,1111055000,37.597352,126.960548,0.0,0.000000,0.0
4,1156054000,37.526698,126.92512,1111056000,37.61164,126.970384,0.0,0.000000,0.0


In [28]:
def plus(data):
    data=data.to_numpy()
    data=data.copy()
    for i in range(len(data)):
        if data[i]<0: data[i]=0
    return data

def minus(data): 
    data=data.to_numpy()
    data=data.copy()
    for i in range(len(data)):
        if data[i]>0: data[i]=0
    return -1*data

In [29]:
df_diff=df_in[['from','from_lat','from_lon','to','to_lat','to_lon']].copy()
df_diff['pop'] = df_in['pop']-df_out['pop']
df_diff['scaled_pop']=df_diff['pop'] / sc
df_diff['scaled_pop_p']=plus(df_diff['scaled_pop'])
df_diff['scaled_pop_m']=minus(df_diff['scaled_pop'])
df_diff['trunc_scaled_pop']=trunc(abs(df_diff['scaled_pop']),alpha)
df_diff['abs_scaled_pop']=abs(df_diff['scaled_pop'])
df_diff['trunc_abs_scaled_pop']=abs(df_diff['trunc_scaled_pop'])
df_diff.head()

,from,from_lat,from_lon,to,to_lat,to_lon,pop,scaled_pop,scaled_pop_p,scaled_pop_m,trunc_scaled_pop,abs_scaled_pop,trunc_abs_scaled_pop
0,1111051500,37.585281,126.968757,1156054000,37.526698,126.92512,0.0,0.000000,0.000000,-0.0,0.00,0.000000,0.00
1,1111053000,37.575267,126.969738,1156054000,37.526698,126.92512,72.0,0.480000,0.480000,-0.0,0.48,0.480000,0.48
2,1111054000,37.586728,126.982915,1156054000,37.526698,126.92512,5.0,0.033333,0.033333,-0.0,0.00,0.033333,0.00
3,1111055000,37.597352,126.960548,1156054000,37.526698,126.92512,0.0,0.000000,0.000000,-0.0,0.00,0.000000,0.00
4,1111056000,37.61164,126.970384,1156054000,37.526698,126.92512,19.0,0.126667,0.126667,-0.0,0.00,0.126667,0.00
